In [1]:
# getting data

In [1]:
import os
import re
import sys
import csv
import random
import requests
import subprocess
from pathlib import Path

from Bio import SeqIO, Align
from Bio.Align import substitution_matrices

import time

In [2]:
soft_dir = Path().resolve().parent / 'soft/vira/vira'
sys.path.append(str(soft_dir))


from classes.txgroup import Transcriptome, Gene, Bundle
from classes.transcript import Transcript, Object
from utils.common import *

In [10]:
base_dir = Path.cwd().parent
data_dir = base_dir / 'data'
outdir = data_dir / 'sequences'
outdir.mkdir(exist_ok=True)

accession_list_fname = data_dir / 'complete_sequences.2022.accessions'
summary_file = data_dir / "download.summary.tsv"

In [4]:
accessions = []
with open(accession_list_fname) as inFP:
    accessions = inFP.read().splitlines()
print(accessions[:5])

['K03455', 'MH705157', 'JQ403028', 'DQ396400', 'DQ676872']


In [16]:
fasta_url_template = "https://www.ncbi.nlm.nih.gov/sviewer/viewer.cgi?db=nuccore&report=fasta&id={}"
gff3_url_template = "https://www.ncbi.nlm.nih.gov/sviewer/viewer.cgi?db=nuccore&report=gff3&id={}"

def download_file(url, save_path):
    try:
        response = requests.get(url, stream=True)
        if response.status_code == 200:
            with open(save_path, 'wb') as file:
                for chunk in response.iter_content(chunk_size=8192):
                    file.write(chunk)
            return True
        else:
            return False
    except requests.exceptions.HTTPError as err:
        print(f"HTTP error occurred: {err}")
        return False
    except Exception as err:
        print(f"An error occurred: {err}")
        return False
    
def verify_genome(qry_fasta_fname, ref_fasta_fname, ref_gtf_fname) -> bool:
    # maps query to the reference
    # finds 3' and 5' clipping
    # checks if clipping includes any annotated regions
    # return False if genome is incomplete

summary_data = []

for acc in accessions:
    fasta_url = fasta_url_template.format(acc)
    gff3_url = gff3_url_template.format(acc)

    fasta_file = outdir / f"{acc}.fasta"
    gff3_file = outdir / f"{acc}.gff3"

    fasta_downloaded = download_file(fasta_url, fasta_file)
    
    gff3_downloaded = download_file(gff3_url, gff3_file)

    summary_data.append([acc, fasta_downloaded, gff3_downloaded])

with open(summary_file, 'w', newline='') as tsvfile:
    writer = csv.writer(tsvfile, delimiter='\t')
    writer.writerow(["accession_id", "fasta", "gff"])
    writer.writerows(summary_data)

SyntaxError: expected ':' (1420445445.py, line 21)

In [14]:
def cleanup_hiv_gtf(input_gtf, output_gtf):
    # get cds chains for each transcript
    seqid = ""
    transcripts = {} # collect all unique CDS chains for each transcript
    tid2gid = {} # map transcript id to gene id
    with open(input_gtf, 'r') as inFP:
        for line in inFP:
            if line.startswith("#"):
                continue
        
            lcs = line.split("\t")

            attrs = extract_attributes(lcs[8])
            
            if lcs[2] == "transcript":
                gid = attrs["gene_id"]
                tid = attrs["transcript_id"]
                tid2gid[tid] = gid
                
            if lcs[2] != "CDS":
                continue

            seqid = lcs[0]
            tid = attrs["transcript_id"]
            if tid not in transcripts:
                transcripts[tid] = set()
            transcripts[tid].add(tuple([lcs[3],lcs[4],lcs[7]])) # start, end, phase

    # collect all unique CDS chains for each gene
    genes = {} # collect all unique CDS chains for each gene
    for tid, cds_chain in transcripts.items():
        gid = tid2gid[tid]
        if gid not in genes:
            genes[gid] = cds_chain
        assert genes[gid] == cds_chain, f"Gene {gid} has different CDS chains for different transcripts"

    # write out standardized GTF file
    with open(output_gtf+".tmp", 'w') as outFP:
        for gid, cds_chain in genes.items():
            for i, (start, end, phase) in enumerate(cds_chain):
                lcs = [seqid, "cleanup", "CDS", start, end, ".", "+", phase, "."]
                attrs = {"gene_id":gid,"transcript_id":gid}
                lcs[8] = to_attribute_string(attrs)
                outFP.write("\t".join(lcs)+"\n")

    # run gffread to generate the final gtf file
    cmd = ["gffread", "-E", "-T", "-o", output_gtf, output_gtf+".tmp"]
    subprocess.run(cmd)

In [15]:
# iterate over the gff files and standardize them:
# - run gffread
# - parse atributes and assign appropriate gene_id and transcript_id

with open(summary_file,"r") as inFP:
    reader = csv.reader(inFP, delimiter="\t")
    # skip header
    next(reader)
    for (accid, fasta, gff) in reader:
        if accid == "K03455": # skip HXB2
            continue
        print(accid)
        
        fasta = bool(fasta)
        gff = bool(gff)
        if not fasta or not gff:
            print(f"Skipping {accid}")
            continue
        
        gff3_file = os.path.join(outdir, f"{accid}.gff3")
        gffread_fname = os.path.join(outdir, f"{accid}.gffread.gtf")
        cmd = ["gffread", "-E", "-T", "-o", gffread_fname, gff3_file]
        subprocess.call(cmd)
        
        gtf_fname = os.path.join(outdir, f"{accid}.gtf")
        
        cleanup_hiv_gtf(gffread_fname, gtf_fname)

MH705157
JQ403028
DQ396400
DQ676872
MH746258
MH746253
KU168256
MH705158
MH705151
MH705153
MH705133
KU168305
MN153491
KP718918
KP718928
MT349418
FJ388893
FJ388903
FJ388909
FJ388925
FJ388938
FJ388942
FJ388943
JF683759
JF683760
JF683767
JF683779
JF683782
JF683783


Command line was:
gffread -E -T -o /home/avaraby1/HIV_Atlas/data/sequences/MH705157.gffread.gtf /home/avaraby1/HIV_Atlas/data/sequences/MH705157.gff3
   .. loaded 10 genomic features from /home/avaraby1/HIV_Atlas/data/sequences/MH705157.gff3
Command line was:
gffread -E -T -o /home/avaraby1/HIV_Atlas/data/sequences/MH705157.gtf /home/avaraby1/HIV_Atlas/data/sequences/MH705157.gtf.tmp
   .. loaded 8 genomic features from /home/avaraby1/HIV_Atlas/data/sequences/MH705157.gtf.tmp
Command line was:
gffread -E -T -o /home/avaraby1/HIV_Atlas/data/sequences/JQ403028.gffread.gtf /home/avaraby1/HIV_Atlas/data/sequences/JQ403028.gff3
   .. loaded 9 genomic features from /home/avaraby1/HIV_Atlas/data/sequences/JQ403028.gff3
Command line was:
gffread -E -T -o /home/avaraby1/HIV_Atlas/data/sequences/JQ403028.gtf /home/avaraby1/HIV_Atlas/data/sequences/JQ403028.gtf.tmp
   .. loaded 9 genomic features from /home/avaraby1/HIV_Atlas/data/sequences/JQ403028.gtf.tmp
Command line was:
gffread -E -T -o /hom

FJ670519
FJ670523
KY496622
MF109677
MF109427
MF109613
MF109623
KT152846
KT152839
KT152840
KT152841
KT152844
AF457052
AF457053
AF457055
AF457066
AF457067
AF457068
AF457069
AF457070
AF457077
AF457079
AF457080
AF457081
AF457083
AF457084
AF457086
AF457089
EU110095
EU110088


Command line was:
gffread -E -T -o /home/avaraby1/HIV_Atlas/data/sequences/FJ670519.gffread.gtf /home/avaraby1/HIV_Atlas/data/sequences/FJ670519.gff3
   .. loaded 9 genomic features from /home/avaraby1/HIV_Atlas/data/sequences/FJ670519.gff3
Command line was:
gffread -E -T -o /home/avaraby1/HIV_Atlas/data/sequences/FJ670519.gtf /home/avaraby1/HIV_Atlas/data/sequences/FJ670519.gtf.tmp
   .. loaded 9 genomic features from /home/avaraby1/HIV_Atlas/data/sequences/FJ670519.gtf.tmp
Command line was:
gffread -E -T -o /home/avaraby1/HIV_Atlas/data/sequences/FJ670523.gffread.gtf /home/avaraby1/HIV_Atlas/data/sequences/FJ670523.gff3
   .. loaded 9 genomic features from /home/avaraby1/HIV_Atlas/data/sequences/FJ670523.gff3
Command line was:
gffread -E -T -o /home/avaraby1/HIV_Atlas/data/sequences/FJ670523.gtf /home/avaraby1/HIV_Atlas/data/sequences/FJ670523.gtf.tmp
   .. loaded 9 genomic features from /home/avaraby1/HIV_Atlas/data/sequences/FJ670523.gtf.tmp
Command line was:
gffread -E -T -o /home

EU110092
EU110094
KT022360
KT022361
KT022363
KT022364
KT022365
KT022367
KT022368
KT022369
KT022370
KT022372
KT022373
KT022374
KT022375
KT022376
KT022377
KT022378
KT022380
KT022381
KT022382
KT022383
FJ623487
FJ623481
FJ623475
FJ623476
FJ623480
FJ623485
FJ623483
FJ623488
FJ623478


...

Command line was:
gffread -E -T -o /home/avaraby1/HIV_Atlas/data/sequences/AY169811.gffread.gtf /home/avaraby1/HIV_Atlas/data/sequences/AY169811.gff3
   .. loaded 9 genomic features from /home/avaraby1/HIV_Atlas/data/sequences/AY169811.gff3
Command line was:
gffread -E -T -o /home/avaraby1/HIV_Atlas/data/sequences/AY169811.gtf /home/avaraby1/HIV_Atlas/data/sequences/AY169811.gtf.tmp
   .. loaded 9 genomic features from /home/avaraby1/HIV_Atlas/data/sequences/AY169811.gtf.tmp
Command line was:
gffread -E -T -o /home/avaraby1/HIV_Atlas/data/sequences/AY169804.gffread.gtf /home/avaraby1/HIV_Atlas/data/sequences/AY169804.gff3
   .. loaded 9 genomic features from /home/avaraby1/HIV_Atlas/data/sequences/AY169804.gff3
Command line was:
gffread -E -T -o /home/avaraby1/HIV_Atlas/data/sequences/AY169804.gtf /home/avaraby1/HIV_Atlas/data/sequences/AY169804.gtf.tmp
   .. loaded 9 genomic features from /home/avaraby1/HIV_Atlas/data/sequences/AY169804.gtf.tmp
Command line was:
gffread -E -T -o /home

JX245015
AJ302646
AJ302647
JN571034
AY169805
AY169814
KF859744
AY532635
GQ324959
DQ017382
DQ017383
GQ324962
GQ324958
MF767262
KY498771
AJ006022
AJ271370
JN572926
HQ179987
KY953207
JQ866001
U42720
AY169968
EF535993
DQ373065
DQ373064
DQ373063
EF535994
DQ373066
FR686511
AF115393
AJ271369


Command line was:
gffread -E -T -o /home/avaraby1/HIV_Atlas/data/sequences/AY169814.gtf /home/avaraby1/HIV_Atlas/data/sequences/AY169814.gtf.tmp
   .. loaded 8 genomic features from /home/avaraby1/HIV_Atlas/data/sequences/AY169814.gtf.tmp
Command line was:
gffread -E -T -o /home/avaraby1/HIV_Atlas/data/sequences/KF859744.gffread.gtf /home/avaraby1/HIV_Atlas/data/sequences/KF859744.gff3
   .. loaded 8 genomic features from /home/avaraby1/HIV_Atlas/data/sequences/KF859744.gff3
Command line was:
gffread -E -T -o /home/avaraby1/HIV_Atlas/data/sequences/KF859744.gtf /home/avaraby1/HIV_Atlas/data/sequences/KF859744.gtf.tmp
   .. loaded 8 genomic features from /home/avaraby1/HIV_Atlas/data/sequences/KF859744.gtf.tmp
Command line was:
gffread -E -T -o /home/avaraby1/HIV_Atlas/data/sequences/AY532635.gffread.gtf /home/avaraby1/HIV_Atlas/data/sequences/AY532635.gff3
   .. loaded 9 genomic features from /home/avaraby1/HIV_Atlas/data/sequences/AY532635.gff3
Command line was:
gffread -E -T -o /home

X52154
AF382828
AF447763
EF394357
DQ374658
JQ768416
JN091691
JN091690
AF103818
FJ424871
FJ424866
FJ424863
MW815971
KP004989
KP004991
KP004990


   .. loaded 9 genomic features from /home/avaraby1/HIV_Atlas/data/sequences/DQ374658.gff3
Command line was:
gffread -E -T -o /home/avaraby1/HIV_Atlas/data/sequences/DQ374658.gtf /home/avaraby1/HIV_Atlas/data/sequences/DQ374658.gtf.tmp
   .. loaded 9 genomic features from /home/avaraby1/HIV_Atlas/data/sequences/DQ374658.gtf.tmp
Command line was:
gffread -E -T -o /home/avaraby1/HIV_Atlas/data/sequences/JQ768416.gffread.gtf /home/avaraby1/HIV_Atlas/data/sequences/JQ768416.gff3
   .. loaded 9 genomic features from /home/avaraby1/HIV_Atlas/data/sequences/JQ768416.gff3
Command line was:
gffread -E -T -o /home/avaraby1/HIV_Atlas/data/sequences/JQ768416.gtf /home/avaraby1/HIV_Atlas/data/sequences/JQ768416.gtf.tmp
   .. loaded 9 genomic features from /home/avaraby1/HIV_Atlas/data/sequences/JQ768416.gtf.tmp
Command line was:
gffread -E -T -o /home/avaraby1/HIV_Atlas/data/sequences/JN091691.gffread.gtf /home/avaraby1/HIV_Atlas/data/sequences/JN091691.gff3
   .. loaded 9 genomic features from /ho